In [1]:
import os
import glob
import pandas as pd

In [2]:
df = pd.read_csv('./outfile.csv',header=1)

In [3]:
df.dropna(inplace=True)
df = df.reset_index(drop=True)
df['Unnamed: 0'].drop(index=1,inplace=True)

In [4]:
df['post_date']= pd.to_datetime(df['post_date'])
df['post_date'] = df['post_date'].dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')

C:\Users\manish-laptop\AppData\Roaming\Python\Python37\site-packages\dateutil\parser\_parser.py:1206: UnknownTimezoneWarning: tzname IST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [5]:
pd.set_option('display.max_colwidth', -1)
df.head()

,Unnamed: 0,post_title,post_text,post_date,category
0,0.0,15th edition of Golden leaf India awards presented,15th edition golden leaf india awards southern tea competition jointly organized united planters association southern india tea board ahmedabad saw 66 shortlisted teas qualified screening panel jury comprising renowned tasters major buyers arun kumar convenor competition said bombay trading corporation five awards kanan devan hills plantations company pvt ltd kdhp united nilgiri tea estates company ltd shared four awards devon plantations industries ltd joonktollee tea industries ltd harrisons malayalam limitedsbu b kodanad estate shared three awards harrisons malayalam limitedsbu two awards waterfall estate pvt ltd woodbriar group shared one award vigneshwar estate tea factory six awards bought leaf factory segment kdhp managing director k mathew abraham said awards company high range region recognition high quality standards maintained company safe sustainable agronomical practices kdhp largest plantation company south india 41 awards years munnar region facilities located besides indian market exports tea europe us russian asian countries,2019-04-30 23:30:00+05:30,agriculture
1,1.0,Multi-state initiative in South India to help paddy farmers,kochi group paddy farmers kerala taken initiative form association linking compatriots southern states provide marketing support ensure fair price produce eliminating middlemen multistate paddy farmers association proposes add 500 core paddy farmers thousands associate members paddy farmers kerala getting fair price produce cost labour expenses gone buy produce directly help marketing said nv giridharan president association based palakkad along alappuzha accounts bulk paddy production state giridharan said farmers southern states also benefit able sell rice kerala without middlemen present rice often sold palakkadan matta premium reddish rice variety kerala states may rice added red colour sell original variety said association sell different types rice five states according giridharan focus online sales hopes start supply starting palakkad registered right majority members kerala states said andhra pradesh tamil nadu major paddy producers southern india association least 100 farmer members five states member one share paying rs 1000 also get one voting right besides others take associate membership paying rs 100 associate members voting rights get dividend association hopes around 5000 members current support price rs 25 50 per kg kerala sufficient farmers meet expenses association share cost beneficial us besides get marketing support states said krishnankutty paddy farmer member association cultivating organic matta rice palakkad according giridharan retired nutritional scientist longterm objective association includes making agriculture financially viable occupation ensuring safe health consumers,2019-04-29 19:30:00+05:30,agriculture
2,2.0,Wheat import duty increased to 40%,new delhi government increased import duty wheat 40 30 bid boost offtake domestically produced grain decision announced friday comes backdrop governments plan sell wheat open market offload excess grain storage units food corporation india state agencies senior official said higher duty made import wheat especially cheaper grains russia unviable flour mills mills purchase overseas pushes price farmers get produce last year able sell 7 million tonnes wheat open sale millers year also likely sell much hike duty would ensure millers wheat purchase benefits indian farmers official told et flour miller said customs duty raised governments open sale wheat millers biscuit makers could affected government plans sell 10 million tonnes wheat 2 million tonnes rice open market oversupply domestic market due backtoback bumper production foodgrains put wheat prices pressure country year india likely produce 99 12 million tonnes grain government first imposed 10 import duty wheat march 2017 november year doubled 20 raise

In [6]:
df.columns

Index(['Unnamed: 0', 'post_title', 'post_text', 'post_date', 'category'], dtype='object')

In [7]:
df['post_text'][0]

'15th edition golden leaf india awards southern tea competition jointly organized united planters association southern india tea board ahmedabad saw 66 shortlisted teas qualified screening panel jury comprising renowned tasters major buyers arun kumar convenor competition said bombay trading corporation five awards kanan devan hills plantations company pvt ltd kdhp united nilgiri tea estates company ltd shared four awards devon plantations industries ltd joonktollee tea industries ltd harrisons malayalam limitedsbu b kodanad estate shared three awards harrisons malayalam limitedsbu two awards waterfall estate pvt ltd woodbriar group shared one award vigneshwar estate tea factory six awards bought leaf factory segment kdhp managing director k mathew abraham said awards company high range region recognition high quality standards maintained company safe sustainable agronomical practices kdhp largest plantation company south india 41 awards years munnar region facilities located besides i

In [8]:
df.isnull().sum()

Unnamed: 0    0
post_title    0
post_text     0
post_date     0
category      0
dtype: int64

In [9]:
df['category'].value_counts()

foreign-trade       48
finance             47
infrastructure      47
agriculture         47
corporate-trends    47
policy              45
indicators          43
Name: category, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

X = df['post_text']  # this time we want to look at the text
y = df['category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(217, 10332)

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(217, 10332)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(217, 10332)

In [14]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [15]:
from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.svm import LinearSVC

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)  

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [16]:
# Form a prediction set
predictions = text_clf.predict(X_test)

In [17]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[15  0  0  1  1  0  0]
 [ 0 11  1  0  0  0  4]
 [ 0  0  8  0  3  1  2]
 [ 0  1  0 16  1  0  0]
 [ 1  0  1  1 11  0  0]
 [ 0  1  0  0  0 12  0]
 [ 0  3  1  0  4  0  7]]


In [18]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

                  precision    recall  f1-score   support

     agriculture       0.94      0.88      0.91        17
corporate-trends       0.69      0.69      0.69        16
         finance       0.73      0.57      0.64        14
   foreign-trade       0.89      0.89      0.89        18
      indicators       0.55      0.79      0.65        14
  infrastructure       0.92      0.92      0.92        13
          policy       0.54      0.47      0.50        15

       micro avg       0.75      0.75      0.75       107
       macro avg       0.75      0.74      0.74       107
    weighted avg       0.76      0.75      0.75       107



In [19]:
print(metrics.accuracy_score(y_test,predictions))

0.7476635514018691


In [20]:
# from sklearn.

In [21]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [22]:
df['scores'] = df['post_text'].apply(lambda text : sid.polarity_scores(text))

In [23]:
df['compound'] = df['scores'].apply(lambda d : d['compound'])

In [24]:
df['comp_score'] = df['compound'].apply(lambda score: 'positive' if score >=0 else 'negative')

In [25]:
df['comp_score'].value_counts()

positive    261
negative    63 
Name: comp_score, dtype: int64

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [27]:
cv = CountVectorizer(max_df=0.9,min_df=2,stop_words='english')

In [28]:
dtm = cv.fit_transform(df['post_text'])

In [29]:
LDA =  LatentDirichletAllocation(n_components=7,random_state=42)

In [30]:
LDA.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=7, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=42, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [31]:
cv.get_feature_names()[500:505]

['ambit', 'ambitions', 'ambitious', 'amc', 'amcs']

In [32]:
len(LDA.components_)

7

In [33]:
LDA.components_.shape

(7, 5848)

In [34]:
len(cv.get_feature_names())

5848

In [35]:
import random
for i in range(10):
    random_word_id = random.randint(0,5848)
    print(cv.get_feature_names()[random_word_id])

accessible
maintains
banking
voted
account
offices
monitors
inherent
pricesensitive
investing


In [36]:
single_topic = LDA.components_[0]

In [37]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([4997, 1785, 2314, ..., 1489, 4159, 4634], dtype=int64)

In [38]:
# Word least representative of this topic
single_topic[1785]

0.14285714436971153

In [39]:
# Word most representative of this topic
single_topic[4634]

108.90128082791664

In [40]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([3065, 1426, 2121, 3430, 4607, 4156, 4663, 1489, 4159, 4634],
      dtype=int64)

In [41]:
top_word_indices = single_topic.argsort()[-10:]

In [42]:
for index in top_word_indices:
    print(cv.get_feature_names()[index])

km
cost
expressway
minister
road
project
said
crore
projects
rs


In [43]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-20:]])
    print('\n')
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['gadkari', 'foundation', 'infrastructure', 'transport', 'new', 'delhi', 'highways', 'highway', 'construction', 'government', 'km', 'cost', 'expressway', 'minister', 'road', 'project', 'said', 'crore', 'projects', 'rs']




THE TOP 15 WORDS FOR TOPIC #1
['steel', 'resolution', 'directors', 'mills', 'ilfs', 'lakh', 'independent', 'companies', 'cent', 'creditors', 'company', 'sugar', 'government', 'rs', 'india', 'production', 'year', 'million', 'said', 'tonnes']




THE TOP 15 WORDS FOR TOPIC #2
['companies', 'year', 'banks', 'gdp', 'need', 'data', 'target', 'sector', 'lakh', 'cent', 'new', 'rbi', 'fiscal', 'bank', 'growth', 'india', 'government', 'crore', 'rs', 'said']




THE TOP 15 WORDS FOR TOPIC #3
['court', 'million', 'report', 'money', 'department', 'new', 'company', 'group', 'banks', 'crore', 'india', 'government', 'scheme', 'companies', 'financial', 'information', 'rs', 'farmers', 'tax', 'said']




THE TOP 15 WORDS FOR TOPIC #4
['farmer', 'bank', '

### Attaching Discovered Topic Labels to Original Articles

In [44]:
dtm

<324x5848 sparse matrix of type '<class 'numpy.int64'>'
	with 46158 stored elements in Compressed Sparse Row format>

In [45]:
dtm.shape

(324, 5848)

In [46]:
len(df)

324

In [47]:
topic_results = LDA.transform(dtm)

In [48]:
topic_results.shape

(324, 7)

In [49]:
topic_results[0]

array([0.00152216, 0.99085753, 0.00152442, 0.00152239, 0.0015238 ,
       0.00152419, 0.0015255 ])

In [50]:
topic_results[0].round(2)

array([0.  , 0.99, 0.  , 0.  , 0.  , 0.  , 0.  ])

In [51]:
topic_results[0].argmax()

1

This means that our model thinks that the first article belongs to topic #1

### Combining with Original Data

In [52]:
df.head()

,Unnamed: 0,post_title,post_text,post_date,category,scores,compound,comp_score
0,0.0,15th edition of Golden leaf India awards presented,15th edition golden leaf india awards southern tea competition jointly organized united planters association southern india tea board ahmedabad saw 66 shortlisted teas qualified screening panel jury comprising renowned tasters major buyers arun kumar convenor competition said bombay trading corporation five awards kanan devan hills plantations company pvt ltd kdhp united nilgiri tea estates company ltd shared four awards devon plantations industries ltd joonktollee tea industries ltd harrisons malayalam limitedsbu b kodanad estate shared three awards harrisons malayalam limitedsbu two awards waterfall estate pvt ltd woodbriar group shared one award vigneshwar estate tea factory six awards bought leaf factory segment kdhp managing director k mathew abraham said awards company high range region recognition high quality standards maintained company safe sustainable agronomical practices kdhp largest plantation company south india 41 awards years munnar region facilities located besides indian market exports tea europe us russian asian countries,2019-04-30 23:30:00+05:30,agriculture,"{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.9909}",0.9909,positive
1,1.0,Multi-state initiative in South India to help paddy farmers,kochi group paddy farmers kerala taken initiative form association linking compatriots southern states provide marketing support ensure fair price produce eliminating middlemen multistate paddy farmers association proposes add 500 core paddy farmers thousands associate members paddy farmers kerala getting fair price produce cost labour expenses gone buy produce directly help marketing said nv giridharan president association based palakkad along alappuzha accounts bulk paddy production state giridharan said farmers southern states also benefit able sell rice kerala without middlemen present rice often sold palakkadan matta premium reddish rice variety kerala states may rice added red colour sell original variety said association sell different types rice five states according giridharan focus online sales hopes start supply starting palakkad registered right majority members kerala states said andhra pradesh tamil nadu major paddy producers southern india association least 100 farmer members five states member one share paying rs 1000 also get one voting right besides others take associate membership paying rs 100 associate members voting rights get dividend association hopes around 5000 members current support price rs 25 50 per kg kerala sufficient farmers meet expenses association share cost beneficial us besides get marketing support states said krishnankutty paddy farmer member association cultivating organic matta rice palakkad according giridharan retired nutritional scientist longterm objective association includes making agriculture financially viable occupation ensuring safe health consumers,2019-04-29 19:30:00+05:30,agriculture,"{'neg': 0.0, 'neu': 0.833, 'pos': 0.167, 'compound': 0.9884}",0.9884,positive
2,2.0,Wheat import duty increased to 40%,new delhi government increased import duty wheat 40 30 bid boost offtake domestically produced grain decision announced friday comes backdrop governments plan sell wheat open market offload excess grain storage units food corporation india state agencies senior official said higher duty made import wheat especially cheaper grains russia unviable flour mills mills purchase overseas pushes price farmers get produce last year able sell 7 million tonnes wheat open sale millers year also likely sell much hike duty would ensure millers wheat purchase benefits indian farmers official told et flour miller said customs duty raised governments open sale wheat millers biscuit makers could affected government plans sell 10 million tonnes wheat 2 million tonnes rice open market oversupply domestic market due backtoback bumper production 

In [53]:
topic_results.argmax(axis=1)

array([1, 2, 1, 1, 1, 2, 3, 2, 1, 6, 2, 5, 6, 4, 3, 1, 1, 6, 2, 6, 3, 4,
       1, 5, 2, 2, 4, 1, 2, 3, 6, 4, 4, 2, 6, 6, 2, 2, 0, 1, 1, 1, 2, 4,
       1, 4, 1, 6, 3, 3, 6, 3, 1, 6, 2, 6, 4, 6, 1, 5, 1, 2, 6, 6, 6, 4,
       5, 6, 4, 3, 2, 6, 3, 1, 5, 2, 6, 2, 1, 5, 5, 2, 5, 2, 2, 6, 5, 2,
       6, 2, 0, 6, 4, 5, 2, 5, 2, 6, 2, 2, 6, 6, 6, 1, 6, 2, 6, 2, 2, 2,
       2, 2, 2, 2, 2, 5, 2, 2, 2, 3, 2, 2, 6, 2, 3, 2, 6, 4, 2, 6, 2, 2,
       2, 2, 2, 5, 2, 5, 6, 1, 6, 5, 6, 3, 2, 1, 1, 6, 2, 1, 3, 1, 6, 6,
       2, 6, 0, 5, 2, 2, 6, 6, 6, 5, 6, 6, 6, 6, 1, 5, 5, 5, 2, 2, 1, 6,
       6, 5, 6, 1, 2, 6, 6, 6, 4, 2, 5, 4, 2, 6, 6, 5, 6, 6, 5, 2, 5, 6,
       2, 5, 6, 6, 2, 2, 6, 2, 2, 6, 2, 2, 5, 2, 2, 6, 2, 2, 2, 2, 2, 5,
       2, 2, 2, 2, 6, 2, 2, 6, 2, 2, 2, 2, 2, 2, 6, 2, 0, 2, 4, 6, 6, 2,
       1, 0, 0, 0, 6, 1, 6, 5, 6, 0, 0, 3, 0, 5, 6, 6, 0, 2, 0, 0, 6, 2,
       0, 0, 3, 0, 2, 2, 0, 3, 0, 4, 6, 0, 4, 2, 6, 2, 4, 1, 4, 3, 6, 5,
       2, 6, 5, 5, 3, 5, 5, 1, 2, 3, 5, 3, 2, 5, 2,

In [54]:
df['Topic'] = topic_results.argmax(axis=1)

In [55]:
df.tail()

,Unnamed: 0,post_title,post_text,post_date,category,scores,compound,comp_score,Topic
319,43.0,MNCs in India may face higher taxes,mumbai rules attributing profits multinational enterprises permanent establishment pe business connection india set significantly change draft report issued thursday night committee appointed central board direct taxes cbdt adopted according tax experts mncs carrying offshore operations india also sales revenue country may find higher profits attributed indian operations prescribed formuladriven approach result higher tax outgo foreign enterprise pe india general refers fixed place business defined tax treaties india tax business profits entity however tax imposed profits attributed operations carried india detailed accounts available determine attribution rule 10 empowers income tax officer indirect apportionment say percentage turnover however lack uniformity approach results litigation report cbdtappointed committee states appropriate attribute profits pe exclusively basis functions assets risks far alone game changer panels report outlines formula calculating profits attributed operations india giving equal weightage assigned sales manpower assets threefactor approach challenge significant manpower deployed india sales revenue also generated lot offshore centres india provide technical services overseas head office group companies could adversely impacted could result higher tax outgo says girish vanvari founder boutique advisory firm transactionsquare however report adds profits derived indian operations already subjected tax india say hands subsidiary deducted apportioned profits floor rate profits derived india set protect revenue interests committee suggests minimum 2 turnover derived deemed profits derived india certain circumstances say foreign enterprise incurring global losses also provides sale taken place india profits apportioned supply activities already taxed hands indian subsidiary may taxes payable foreign enterprise panel also considered emerging business models business models users contribute significantly profits foreign enterprise user base could taken account purpose profit attribution fourth factor apportionment addition sales manpower assets weightage range 1020 assigned user base points nangia advisors director sandeep jhunjhunwala,2019-04-19 15:30:00+05:30,policy,"{'neg': 0.044, 'neu': 0.782, 'pos': 0.174, 'compound': 0.9891}",0.9891,positive,4
320,44.0,View: India’s growth numbers are not reflected by other indicators,kaushik basu paradoxical according latest government data indias gdp growth rate 6 6 though weak comparison indias performance last decade half global comparative terms commendable however terms several critical indicators performance economy outright disappointing latest growth number february 2019 core sector 2 1 index industrial production 0 1 export growth last five years zero rarely happened india 1991 indias savings investments rates crossed 30 2003 went 35 2008 making india look like fastgrowing economies east asia slid back 30 agriculture sector floundering finally india watchers concerned countrys jobs situation release official data stopped detailed studies azim premji university centre monitoring indian economy piecemeal evidence job announcements numbers applicants make clear situation grim square indias commendable growth indicators terms pure accounting explained growing inequality growth happening concentrated top bulk society working classes farmers small businesses barely inching forward come india important recognise several good policy initiatives also mistakes demonetisation november 2016 virtual consensus among experts around world blunder big negative effect small businesses workers farmers leaving rich virtually unaffected interestingly sharpest indictment comes finance ministrys economic survey 201617 lists countries similar sudden demonetisations since 1982 north korea venezuela myanmar iraq russia ussr ghana brazil cyprus list mak